In [60]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import pickle 
import json

import dlc_bci as bci
from helpers import *

import numpy as np

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
import matplotlib.pyplot as plt

torch.manual_seed(1)
torch.set_num_threads(4)
np.random.seed(1)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Importing the data

In [43]:
# Subset of data sampled at 100Hz
train_input , train_target = bci.load(root = './data_bci')
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

Train input : <class 'torch.FloatTensor'> torch.Size([316, 28, 50])
Train target : <class 'torch.LongTensor'> torch.Size([316])
Test input : <class 'torch.FloatTensor'> torch.Size([100, 28, 50])
Test target : <class 'torch.LongTensor'> torch.Size([100])


In [44]:
# Full data sampled at 1Khz
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

Train input : <class 'torch.FloatTensor'> torch.Size([316, 28, 500])
Train target : <class 'torch.LongTensor'> torch.Size([316])
Test input : <class 'torch.FloatTensor'> torch.Size([100, 28, 500])
Test target : <class 'torch.LongTensor'> torch.Size([100])


In [45]:
train_input[0]


  30.8000   33.1000   37.9000  ...    76.9000   75.9000   72.3000
 -22.6000  -19.2000  -11.8000  ...    26.8000   22.9000   16.4000
  11.2000   16.7000   26.4000  ...    64.1000   62.0000   55.9000
             ...                ⋱                ...             
   0.5000    0.5000    0.4000  ...    45.1000   46.2000   46.6000
  11.0000   10.6000    9.8000  ...    41.7000   41.5000   40.9000
  -9.9000  -10.0000  -10.3000  ...    32.9000   33.8000   32.8000
[torch.FloatTensor of size 28x500]

In [46]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

### Preprocessing the data
Normalizing :

In [47]:
train_mean = train_input.mean(2).mean(0).unsqueeze(1).expand(-1,train_input.size(2))
train_std = train_input.std(2).std(0).unsqueeze(1).expand(-1,train_input.size(2))

In [48]:
test_mean = test_input.mean(2).mean(0).unsqueeze(1).expand(-1,test_input.size(2))
test_std = test_input.std(2).std(0).unsqueeze(1).expand(-1,test_input.size(2))

In [49]:
# normalize mean to 0
train_input.sub_(train_mean)
test_input.sub_(test_mean)

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


Window slicing for data augmentation :

In [50]:
seq = [ train_input[:,:,i::10] for i in range(10) ]
train_input = torch.cat(seq)

In [51]:
train_target = torch.cat([train_target]*10)

In [52]:
seq = [ test_input[:,:,i::10] for i in range(10) ]
test_input = torch.cat(seq)

In [53]:
test_target = torch.cat([test_target]*10)

In [54]:
train_input


( 0  ,.,.) = 
 -1.1184e+00 -6.6277e-01 -1.1995e+00  ...   4.5434e-01 -2.1655e-01  3.7009e-01
 -6.8330e-01  7.6446e-05 -7.4883e-01  ...   1.0049e+00  8.7448e-02  1.0891e+00
 -1.1344e+00 -6.4085e-02 -1.1812e+00  ...   9.9686e-01 -3.5741e-01  1.2122e+00
                 ...                   ⋱                   ...                
 -3.4267e-01 -2.2721e-01 -7.1193e-02  ...   8.4621e-01  8.3997e-01  7.6820e-01
 -3.7571e-01 -1.5416e-01  2.5774e-01  ...   9.5359e-01  9.5359e-01  6.4467e-01
 -5.9022e-01 -3.2186e-01 -5.3502e-02  ...   8.7327e-01  8.9511e-01  6.1739e-01

( 1  ,.,.) = 
  2.1550e+00  2.1269e+00  2.4670e+00  ...   3.4825e-01  7.7262e-01  5.7604e-01
  5.4927e-01  4.3762e-02  6.2416e-01  ...  -1.2356e+00 -3.5877e-01 -1.0234e+00
 -7.3810e-01 -1.1999e+00 -7.4434e-01  ...  -2.0424e+00 -1.2623e+00 -1.8958e+00
                 ...                   ⋱                   ...                
 -9.4491e-01 -8.7938e-01 -1.2351e+00  ...  -1.3225e+00 -9.8548e-01 -8.4194e-01
 -5.2861e-01 -5.3797e-

Splitting training and validation sets :

In [55]:
split = 2500

In [56]:
indices = np.arange(train_input.size(0))
np.random.shuffle(indices)

In [57]:
full_input = train_input.clone()
train_input = full_input[indices[:split].tolist()]
validation_input = full_input[indices[split:].tolist()]

In [58]:
full_target = train_target.clone()
train_target = full_target[indices[:split].tolist()]
validation_target = full_target[indices[split:].tolist()]

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_reg = LinearRegression()

In [ ]:
linear_reg.fit(train_input.view(train_input.size(0),-1),train_target)

In [ ]:
linear_pred_train = discrete_predictions(torch.FloatTensor(linear_reg.predict(train_input.view(train_input.size(0),-1))))

In [ ]:
print(100*((linear_pred_train - train_target).abs().sum()/train_input.size(0)),"% train error")

In [ ]:
linear_pred_test = discrete_predictions(torch.FloatTensor(linear_reg.predict(test_input.view(test_input.size(0),-1))))

In [ ]:
print(100*((linear_pred_test - test_target).abs().sum()/test_input.size(0)),"% test error")

## Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_reg = Ridge()

In [ ]:
ridge_reg.fit(train_input.view(train_input.size(0),-1),train_target)

In [ ]:
ridge_pred_train = discrete_predictions(torch.FloatTensor(ridge_reg.predict(train_input.view(train_input.size(0),-1))))

In [ ]:
print(100*((ridge_pred_train - train_target).abs().sum()/train_input.size(0)),"% train error")

In [ ]:
ridge_pred_test = discrete_predictions(torch.FloatTensor(ridge_reg.predict(test_input.view(test_input.size(0),-1))))

In [ ]:
print(100*((ridge_pred_test - test_target).abs().sum()/test_input.size(0)),"% test error")

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_reg = LogisticRegression()

In [ ]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

In [ ]:
logistic_pred_train = torch.LongTensor(logistic_reg.predict(train_input.view(train_input.size(0),-1)))

In [ ]:
print(100*((logistic_pred_train - train_target).abs().sum()/train_input.size(0)),"% train error")

In [ ]:
logistic_pred_test = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [ ]:
print(100*((logistic_pred_test - test_target).abs().sum()/test_input.size(0)),"% test error")

## Neural Networks

### Basic MLP

In [ ]:
MLP = nn.Sequential(
        nn.Linear(1400, 140),
        nn.Tanh(),
        nn.Linear(140, 28),
        nn.Tanh(),
        nn.Linear(28, 2)
    )

In [ ]:
train_model(MLP, Variable(full_input.view(full_input.size(0),-1)), Variable(full_target), 40)

In [ ]:
train_error_mlp = compute_nb_errors(MLP, Variable(full_input.view(full_input.size(0),-1)), Variable(full_target))
print(100*(train_error_mlp/full_input.size(0)),'% training error')

In [ ]:
test_error_mlp = compute_nb_errors(MLP, Variable(test_input.view(test_input.size(0),-1)), Variable(test_target))
print(100*(test_error_mlp/test_input.size(0)),'% test error')

### Basic CNN

In [ ]:
class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3)
        self.fc1 = nn.Linear(128, 28)
        self.fc2 = nn.Linear(28, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=(3,2), stride=(3,2)))
        x = F.tanh(self.conv3(x))
        x = F.tanh(self.fc1(x.view(x.size(0),-1)))
        x = self.fc2(x)
        return x

In [ ]:
cnn = Basic_CNN()

In [ ]:
train_model(cnn, Variable(full_input.unsqueeze(1)), Variable(full_target), 40)

In [ ]:
train_error_cnn = compute_nb_errors(cnn, Variable(full_input.unsqueeze(1)), Variable(full_target))
print(100*(train_error_cnn/full_input.size(0)),'% training error')

In [ ]:
test_error_cnn = compute_nb_errors(cnn, Variable(test_input.unsqueeze(1)), Variable(test_target))
print(100*(test_error_cnn/test_input.size(0)),'% test error')

### CNN w 1D filters

In [ ]:
class CNN_1D(nn.Module):
    def __init__(self):
        super(CNN_1D, self).__init__()
        self.conv1 = nn.Conv1d(28, 28, kernel_size=3)
        self.conv2 = nn.Conv1d(28, 14, kernel_size=3)
        self.conv3 = nn.Conv1d(14, 1, kernel_size=3)
        self.fc1 = nn.Linear(9, 36)
        self.fc2 = nn.Linear(36, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool1d(self.conv1(x), kernel_size=2, stride=2))
        x = F.tanh(F.max_pool1d(self.conv2(x), kernel_size=2, stride=2))
        x = F.tanh(self.conv3(x))
        x = self.fc1(x.squeeze(1))
        x = self.fc2(F.tanh(x))
        return x

In [ ]:
cnn_1D = CNN_1D()
train_model(cnn_1D, Variable(full_input), Variable(full_target), 40)

In [ ]:
train_error_cnn_1D = compute_nb_errors(cnn_1D, Variable(full_input), Variable(full_target))
print(100*(train_error_cnn_1D/full_input.size(0)),'% training error')

In [ ]:
test_error_cnn_1D = compute_nb_errors(cnn_1D, Variable(test_input), Variable(test_target))
print(100*(test_error_cnn_1D/test_input.size(0)),'% test error')

### previous + dropout

In [ ]:
class CNN_dropout(nn.Module):
    def __init__(self, dropout=0.5):
        super(CNN_dropout, self).__init__()
        self.conv1 = nn.Conv1d(28, 28, kernel_size=3)
        self.conv2 = nn.Conv1d(28, 14, kernel_size=3)
        self.conv3 = nn.Conv1d(14, 1, kernel_size=3)
        self.fc1 = nn.Linear(9, 30)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(30, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool1d(self.conv1(x), kernel_size=2, stride=2))
        x = F.tanh(F.max_pool1d(self.conv2(x), kernel_size=2, stride=2))
        x = F.tanh(self.conv3(x))
        x = self.fc1(x.squeeze(1))
        x = self.dropout(x)
        x = self.fc2(F.tanh(x))
        return x

In [ ]:
cnn_drop = CNN_dropout()
train_model(cnn_drop, Variable(full_input), Variable(full_target), 40)

In [ ]:
train_error_cnn_drop = compute_nb_errors(cnn_drop, Variable(full_input), Variable(full_target))
print(100*(train_error_cnn_drop/full_input.size(0)),'% training error')

In [ ]:
test_error_cnn_drop = compute_nb_errors(cnn_drop, Variable(test_input), Variable(test_target))
print(100*(test_error_cnn_drop/test_input.size(0)),'% test error')

## Batch normalization

In [ ]:
class CNN_batchnorm(nn.Module):
    def __init__(self):
        super(CNN_batchnorm, self).__init__()
        self.conv1 = nn.Conv1d(28, 28, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(28)
        self.conv2 = nn.Conv1d(28, 14, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(14)
        self.conv3 = nn.Conv1d(14, 1, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(1)
        self.fc1 = nn.Linear(9, 30)
        self.bn4 = nn.BatchNorm1d(30)
        self.fc2 = nn.Linear(30, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool1d(self.bn1(self.conv1(x)), kernel_size=2, stride=2))
        x = F.tanh(F.max_pool1d(self.bn2(self.conv2(x)), kernel_size=2, stride=2))
        x = F.tanh(self.bn3(self.conv3(x)))
        x = self.fc1(x.squeeze(1))
        x = self.fc2(F.tanh(self.bn4(x)))
        return x

In [ ]:
cnn_norm = CNN_batchnorm()
train_model(cnn_norm, Variable(full_input), Variable(full_target), 40)

In [ ]:
train_error_cnn_norm = compute_nb_errors(cnn_norm, Variable(full_input), Variable(full_target))
print(100*(train_error_cnn_norm/full_input.size(0)),'% training error')

In [ ]:
test_error_cnn_norm = compute_nb_errors(cnn_norm, Variable(test_input), Variable(test_target))
print(100*(test_error_cnn_norm/test_input.size(0)),'% test error')

## Both dropout and batchnorm

In [ ]:
class CNN_both(nn.Module):
    def __init__(self, dropout=0.5):
        super(CNN_both, self).__init__()
        self.conv1 = nn.Conv1d(28, 28, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(28)
        self.conv2 = nn.Conv1d(28, 14, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(14)
        self.conv3 = nn.Conv1d(14, 1, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(1)
        self.fc1 = nn.Linear(9, 30)
        self.bn4 = nn.BatchNorm1d(30)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(30, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool1d(self.bn1(self.conv1(x)), kernel_size=2, stride=2))
        x = F.tanh(F.max_pool1d(self.bn2(self.conv2(x)), kernel_size=2, stride=2))
        x = F.tanh(self.bn3(self.conv3(x)))
        x = self.fc1(x.squeeze(1))
        x = self.dropout(self.bn4(x))
        x = self.fc2(F.tanh(x))
        return x

In [ ]:
cnn_both = CNN_both()
train_model(cnn_both, Variable(full_input), Variable(full_target), 40)

In [ ]:
train_error_cnn_both = compute_nb_errors(cnn_both, Variable(full_input), Variable(full_target))
print(100*(train_error_cnn_both/full_input.size(0)),'% training error')

In [ ]:
test_error_cnn_both = compute_nb_errors(cnn_both, Variable(test_input), Variable(test_target))
print(100*(test_error_cnn_both/test_input.size(0)),'% test error')

## Grid search

In [59]:
def find_best_params(network, train_input, train_target, val_input, val_target, learning_rates, nb_epochs_total, epoch_step, mini_batch_size):
    res = []
    best_error = val_input.size(0)+1
    #print("MLP")
    print("MODEL", network().__class__.__name__)
    print("="*18)
    for lr in learning_rates: 
        print("  Learning rate", lr, ":")
        print("-"*23)
        epoch_acc = []
        torch.manual_seed(1)
        model = network()
        #model = nn.Sequential(
        #    nn.Linear(1400, 140),
        #    nn.Tanh(),
        #    nn.Linear(140, 28),
        #    nn.Tanh(),
        #    nn.Linear(28, 2)
        #)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        for e in range(nb_epochs_total):
            sum_loss = 0
            model.train()
            for b in range(0, train_input.size(0), mini_batch_size):
                output = model(train_input.narrow(0, b, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
                sum_loss = sum_loss + loss.data[0]
                model.zero_grad()
                loss.backward()
                optimizer.step()
                
            if (e+1)%epoch_step == 0 or e==0:
                validation_error = compute_nb_errors(model, val_input, val_target)
                if (e+1)%(10*epoch_step) == 0 or e==0:
                    print("    Epoch {:>4} : loss = {:1.8f} | validation error = {:>4}".format(e+1, sum_loss, validation_error))
                epoch_acc.append(100*(validation_error/val_input.size(0)))
                if (validation_error < best_error) or ( (validation_error == best_error) and ((e+1) < best_epoch) ):
                    best_error = validation_error
                    best_epoch = e+1
                    best_lr = lr
                    
        res.append(epoch_acc)
    print("Done.")
    return res, best_epoch, best_lr, best_error

MLP test :

In [70]:
learning_rates = [0.001, 0.0025, 0.005, 0.0075, 0.01]
nb_epochs_total = 500
epoch_step = 5
mini_batch_size = 250

In [71]:
all_results, best_epoch, best_lr, best_error = find_best_params(MLP, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target), Variable(validation_input.view(validation_input.size(0),-1)), Variable(validation_target), learning_rates, nb_epochs_total, epoch_step, mini_batch_size)
json_res = {"model":"MLP",   "learning_rates":learning_rates, "nb_epochs_total":nb_epochs_total, "epoch_step":epoch_step, "mini_batch_size":mini_batch_size, "best_epoch":best_epoch, "best_lr":best_lr, "best_error":best_error, "results":all_results}
json.dump(json_res,open('results/adam/'+json_res["model"]+'.json','w'))

TypeError: forward() missing 1 required positional argument: 'input'

Basic CNN test :

In [72]:
learning_rates = [0.001, 0.0025, 0.005, 0.0075, 0.01]
nb_epochs_total = 100
epoch_step = 5
mini_batch_size = 250

In [73]:
all_results, best_epoch, best_lr, best_error = find_best_params(Basic_CNN, Variable(train_input.unsqueeze(1)), Variable(train_target), Variable(validation_input.unsqueeze(1)), Variable(validation_target), learning_rates, nb_epochs_total, epoch_step, mini_batch_size)
json_res = {"model":"Basic_CNN",   "learning_rates":learning_rates, "nb_epochs_total":nb_epochs_total, "epoch_step":epoch_step, "mini_batch_size":mini_batch_size, "best_epoch":best_epoch, "best_lr":best_lr, "best_error":best_error, "results":all_results}
json.dump(json_res,open('results/adam/'+json_res["model"]+'.json','w'))

MODEL Basic_CNN
  Learning rate 0.001 :
-----------------------
    Epoch    1 : loss = 6.88160986 | validation error =  293
    Epoch   50 : loss = 0.01517187 | validation error =    0
    Epoch  100 : loss = 0.00309679 | validation error =    0
  Learning rate 0.0025 :
-----------------------
    Epoch    1 : loss = 6.95688170 | validation error =  331
    Epoch   50 : loss = 0.00247078 | validation error =    0
    Epoch  100 : loss = 0.00063537 | validation error =    0
  Learning rate 0.005 :
-----------------------
    Epoch    1 : loss = 7.02459210 | validation error =  298
    Epoch   50 : loss = 0.00054406 | validation error =    0
    Epoch  100 : loss = 0.00019461 | validation error =    0
  Learning rate 0.0075 :
-----------------------
    Epoch    1 : loss = 6.98603934 | validation error =  291
    Epoch   50 : loss = 0.00021192 | validation error =    0
    Epoch  100 : loss = 0.00008684 | validation error =    0
  Learning rate 0.01 :
-----------------------
    Epoch  

All the 4 other models :

In [66]:
learning_rates = [0.001, 0.0025, 0.005, 0.0075, 0.01]
nb_epochs_total = 500
epoch_step = 5
mini_batch_size = 250

models = [ CNN_1D, CNN_dropout, CNN_batchnorm, CNN_both ]

In [67]:
for m in models:
    all_results, best_epoch, best_lr, best_error = find_best_params(m, Variable(train_input), Variable(train_target), Variable(validation_input), Variable(validation_target), learning_rates, nb_epochs_total, epoch_step, mini_batch_size)
    print("")
    json_res = {"model":m().__class__.__name__,   "learning_rates":learning_rates, "nb_epochs_total":nb_epochs_total, "epoch_step":epoch_step, "mini_batch_size":mini_batch_size, "best_epoch":best_epoch, "best_lr":best_lr, "best_error":best_error, "results":all_results}
    json.dump(json_res,open('results/adam/'+json_res["model"]+'.json','w'))

MODEL CNN_1D
  Learning rate 0.001 :
-----------------------
    Epoch    1 : loss = 6.86664402 | validation error =  275
    Epoch   50 : loss = 0.52575323 | validation error =   11
    Epoch  100 : loss = 0.02483570 | validation error =    1
    Epoch  150 : loss = 0.02050034 | validation error =    6
    Epoch  200 : loss = 0.01125845 | validation error =    1
    Epoch  250 : loss = 0.00518089 | validation error =    1
    Epoch  300 : loss = 0.00303317 | validation error =    1
    Epoch  350 : loss = 0.00194533 | validation error =    0
    Epoch  400 : loss = 0.00131074 | validation error =    0
    Epoch  450 : loss = 0.00091051 | validation error =    0
    Epoch  500 : loss = 0.00064559 | validation error =    0
  Learning rate 0.0025 :
-----------------------
    Epoch    1 : loss = 6.82721412 | validation error =  244
    Epoch   50 : loss = 0.02413524 | validation error =    1
    Epoch  100 : loss = 0.00222192 | validation error =    1
    Epoch  150 : loss = 0.00098169 |

    Epoch  100 : loss = 0.00175696 | validation error =    1
    Epoch  150 : loss = 0.00074541 | validation error =    1
    Epoch  200 : loss = 0.00040033 | validation error =    1
    Epoch  250 : loss = 0.00024086 | validation error =    1
    Epoch  300 : loss = 0.00015497 | validation error =    1
    Epoch  350 : loss = 0.00010393 | validation error =    1
    Epoch  400 : loss = 0.00007117 | validation error =    1
    Epoch  450 : loss = 0.00005036 | validation error =    1
    Epoch  500 : loss = 0.00003594 | validation error =    1
  Learning rate 0.005 :
-----------------------
    Epoch    1 : loss = 6.79651618 | validation error =  314
    Epoch   50 : loss = 0.00135912 | validation error =    1
    Epoch  100 : loss = 0.00040913 | validation error =    0
    Epoch  150 : loss = 0.00019286 | validation error =    0
    Epoch  200 : loss = 0.00010925 | validation error =    0
    Epoch  250 : loss = 0.00006809 | validation error =    0
    Epoch  300 : loss = 0.00004498 | 

Test results :

In [64]:
torch.manual_seed(1)
model = nn.Sequential(
            nn.Linear(1400, 140),
            nn.Tanh(),
            nn.Linear(140, 28),
            nn.Tanh(),
            nn.Linear(28, 2)
        )
json_res = json.load(open('results/adam/MLP.json','r'))
print("MLP")
print("For", json_res['best_epoch'], "epochs, with a learning rate of", json_res['best_lr'])
train_model(model, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target), json_res['mini_batch_size'], nb_epochs=json_res['best_epoch'], learning_rate=json_res['best_lr'], verbose=False)
test_error = compute_nb_errors(model, Variable(test_input.view(test_input.size(0),-1)), Variable(test_target))
print(100*(test_error/test_input.size(0)))
json_res["test_error"] = 100*(test_error/test_input.size(0))
json.dump(json_res, open('results/adam/MLP.json','w'))

MLP
For 25 epochs, with a learning rate of 0.001
50.0


In [75]:
torch.manual_seed(1)
model = Basic_CNN()
json_res = json.load(open('results/adam/Basic_CNN.json','r'))
print("Basic CNN")
print("For", json_res['best_epoch'], "epochs, with a learning rate of", json_res['best_lr'])
train_model(model, Variable(train_input.unsqueeze(1)), Variable(train_target), json_res['mini_batch_size'], nb_epochs=json_res['best_epoch'], learning_rate=json_res['best_lr'], verbose=False)
test_error = compute_nb_errors(model, Variable(test_input.unsqueeze(1)), Variable(test_target))
print(100*(test_error/test_input.size(0)))
json_res["test_error"] = 100*(test_error/test_input.size(0))
json.dump(json_res, open('results/adam/Basic_CNN.json','w'))

Basic CNN
For 20 epochs, with a learning rate of 0.0025
32.0


In [92]:
error_rates = []
for m in models:
    torch.manual_seed(1)
    json_res = json.load(open('results/adam/'+m().__class__.__name__+'.json','r'))
    print(json_res["model"])
    print("For", json_res['best_epoch'], "epochs, with a learning rate of", json_res['best_lr'])
    acc_error = []
    for i in range(20):
        print("    iteration", i+1)
        model = m()
        train_model(model, Variable(train_input), Variable(train_target), json_res['mini_batch_size'], nb_epochs=json_res['best_epoch'], learning_rate=json_res['best_lr'], verbose=False)
        acc_error.append(compute_nb_errors(model, Variable(test_input), Variable(test_target)))
    test_error = np.mean(acc_error)
    print(100*(test_error/test_input.size(0)))
    json_res["test_error"] = 100*(test_error/test_input.size(0))
    json.dump(json_res, open('results/adam/'+model.__class__.__name__+'.json','w'))
    error_rates.append(100*(test_error/test_input.size(0)))
    print("="*10)

CNN_1D
For 20 epochs, with a learning rate of 0.01
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
    iteration 16
    iteration 17
    iteration 18
    iteration 19
    iteration 20
28.389999999999997
CNN_dropout
For 25 epochs, with a learning rate of 0.01
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
    iteration 16
    iteration 17
    iteration 18
    iteration 19
    iteration 20
31.855
CNN_batchnorm
For 35 epochs, with a learning rate of 0.0075
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
 

In [ ]:
plt.figure(1)

colormap = 'Blues'
normal_xticks = normal_results['nb_epochs']
normal_yticks = normal_results['learning_rates']

plt.subplot(221)
plt.imshow(np.array(normal_results['results'][0]).T, cmap=colormap)
plt.xlabel('Nb of epochs')
plt.ylabel('Learning rate')
plt.xticks(range(len(normal_xticks)), normal_xticks)
plt.yticks(range(len(normal_yticks)), normal_yticks)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Without dropout layer and batch size 20')

plt.subplot(222)
plt.imshow(np.array(normal_results['results'][1]).T, cmap=colormap)
plt.xlabel('Nb of epochs')
plt.ylabel('Learning rate')
plt.xticks(range(len(normal_xticks)), normal_xticks)
plt.yticks(range(len(normal_yticks)), normal_yticks)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Without dropout layer and batch size 40')

dropout_xticks = dropout_results['nb_epochs']
dropout_yticks = dropout_results['learning_rates']

plt.subplot(223)
plt.imshow(dropout_results['results'][0], cmap=colormap)
plt.xlabel('Nb of epochs')
plt.ylabel('Learning rate')
plt.xticks(range(len(dropout_xticks)), dropout_xticks)
plt.yticks(range(len(dropout_yticks)), dropout_yticks)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('With dropout layer and batch size 20')

plt.subplot(224)
plt.imshow(dropout_results['results'][1], cmap=colormap)
plt.xlabel('Nb of epochs')
plt.ylabel('Learning rate')
plt.xticks(range(len(dropout_xticks)), dropout_xticks)
plt.yticks(range(len(dropout_yticks)), dropout_yticks)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('With dropout layer and batch size 40')

plt.tight_layout()
plt.savefig("results_1.png")
plt.show()